In [42]:
import pprint
from tatsu import parse
from tabulate import tabulate
from IPython.display import HTML, display

In [9]:
GRAMMAR = '''
    start = expression $ ;
    
    trash = /\w+/;

    number 
        = /\d+/ 
        ;
        
    factor
        = number
        ;

    expression
        =
        | expression expOperator term
        | trash term
        | trash expression
        ;
    
    term
        =
        | term termOperator factor
        | factor
        ;

    expOperator
        =
        | plusOperator
        | minusOperator
        | trash expOperator
        ;
        
    termOperator
        =
        | multiplicationOperator
        | divisionOperator
        | trash termOperator
        ;
        
    plusOperator
        =
        | 'plus'
        | 'dodać'
        ;
    
    minusOperator
        =
        | 'minus'
        | 'odjąć'
        ;
        
    multiplicationOperator
        = 'razy'
        ;
    
    divisionOperator
        =
        | 'przez'
        | 'dzielone przez'
        | 'nad'
        ;
        
'''

In [10]:
operatorMap = {
    "plus": "+",
    "dodać": "+",
    "minus": "-",
    "odjąć": "-",
    "razy": "*",
    "przez": "/",
    "dzielone przez": "/"
}

In [173]:
def text2int(textnum, numwords={}):
    if not numwords:
      units = [
        "zero", "jeden", "dwa", "trzy", "cztery", "pięć", "sześć", "siedem", "osiem", "dziewięć",
          "dziesięć", "jedenaście", "dwanaście", "trzynaście", "czternaście", "piętnaście", "szesnaście",
          "siedemnaście", "osiemnaście", "dziewiętnaście"
      ]

      tens = ["", "", "dwadzieścia", "trzydzieści", "czterdzieści", "pięćdziesiąć", "sześćdziesiąt", 
              "siedemdziesiąt", "osiemdziesiąt", "dziewięćdziesiąt"]

      numwords["i"] = (1, 0)
      for idx, word in enumerate(units):    numwords[word] = (1, idx)
      for idx, word in enumerate(tens):     numwords[word] = (1, idx * 10)

    current = result = 0
    for word in textnum.split():
        if word not in numwords:
          raise Exception("Illegal word: " + word)

        scale, increment = numwords[word]
        current = current * scale + increment
        if scale > 100:
            result += current
            current = 0

    return result + current

In [174]:
def valid_text2int(word):
    units = [
        "zero", "jeden", "dwa", "trzy", "cztery", "pięć", "sześć", "siedem", "osiem", "dziewięć",
          "dziesięć", "jedenaście", "dwanaście", "trzynaście", "czternaście", "piętnaście", "szesnaście",
          "siedemnaście", "osiemnaście", "dziewiętnaście"]

    tens = ["", "", "dwadzieścia", "trzydzieści", "czterdzieści", "pięćdziesiąć", "sześćdziesiąt", 
            "siedemdziesiąt", "osiemdziesiąt", "dziewięćdziesiąt"]
        
    if word in units or word in tens:
        return True
    else:
        return False

In [180]:
def expression2int(expression):
    expressionWithInts = ""
    partitionedExpression = expression.split(" ")
    i = 0 
    
    while i < len(partitionedExpression):
        if(valid_text2int(partitionedExpression[i])):
            numberToConvert = ""
            while i<len(partitionedExpression) and validText2Int(partitionedExpression[i]):
                numberToConvert += partitionedExpression[i]
                numberToConvert += " "
                i += 1
            expressionWithInts += str(text2int(numberToConvert))
            i -= 1
        else:
            expressionWithInts += partitionedExpression[i]
            
        expressionWithInts += " "
        i += 1
        
    return expressionWithInts

In [11]:
def process(e):
    if e in ['+', '-', '*', '/']:
        return e
    if e.isdigit():
        return e
    if e in operatorMap.keys():
        return operatorMap.get(e)
    else:
        return ''

In [12]:
def debugEvaluate(e):
    result = ''
    print('evaluate:', e)
    for subexpression in e:
        if type(subexpression) is not str:
            partial = debugEvaluate(subexpression)
            result += partial
        else:
            result += process(subexpression)
    print('result:', result)
    return result

def evaluate(e):
    result = ''
    for subexpression in e:
        if type(subexpression) is not str:
            partial = evaluate(subexpression)
            result += partial
        else:
            result += process(subexpression)
    return result

In [13]:
expression = 'policz proszę 5 razy 3 i dodać 88 razy 2'
parsedExpression = parse(GRAMMAR, expression)
pprint.pprint(parsedExpression, indent=2, width=20)

[ 'policz',
  [ 'proszę',
    [ [ '5',
        'razy',
        '3'],
      [ 'i',
        [ 'dodać',
          [ '88',
            'razy']]],
      '2']]]


In [14]:
evaluatedExpression = debugEvaluate(parsedExpression)

evaluate: ['policz', ['proszę', [['5', 'razy', '3'], ['i', ['dodać', ['88', 'razy']]], '2']]]
evaluate: ['proszę', [['5', 'razy', '3'], ['i', ['dodać', ['88', 'razy']]], '2']]
evaluate: [['5', 'razy', '3'], ['i', ['dodać', ['88', 'razy']]], '2']
evaluate: ['5', 'razy', '3']
result: 5*3
evaluate: ['i', ['dodać', ['88', 'razy']]]
evaluate: ['dodać', ['88', 'razy']]
evaluate: ['88', 'razy']
result: 88*
result: +88*
result: +88*
result: 5*3+88*2
result: 5*3+88*2
result: 5*3+88*2


In [15]:
print(evaluatedExpression)

5*3+88*2


In [16]:
eval(evaluatedExpression)

191

In [184]:
def calculate(expression):
    expression = expression2int(expression)
    parsedExpression = parse(GRAMMAR, expression)
    evaluatedExpression = evaluate(parsedExpression)
    result = eval(evaluatedExpression)
    return result

In [200]:
testSentences = [
    'policz proszę pięć razy trzy i dodać osiemdziesiąt osiem razy dwa',
    'ile jest dwa plus dwa razy dwa',
    'ile jest dziesięć plus sześć przez dwa',
    'dwa razy trzy',
    'dwa plus dwa dodać dwa',
    'dwa plus dwa',
    'dziesięć przez dwa',
    'weź mnie policz trzy razy sześć przez dwa dodać osiem razy pięć'
]

In [201]:
result = [[sentence, calculate(sentence)] for sentence in testSentences]

In [202]:
display(HTML(tabulate(result, tablefmt='html', headers=["Sentence", "Evaluation"])))

Sentence,Evaluation
policz proszę pięć razy trzy i dodać osiemdziesiąt osiem razy dwa,191
ile jest dwa plus dwa razy dwa,6
ile jest dziesięć plus sześć przez dwa,13
dwa razy trzy,6
dwa plus dwa dodać dwa,6
dwa plus dwa,4
dziesięć przez dwa,5
weź mnie policz trzy razy sześć przez dwa dodać osiem razy pięć,49
